In [ ]:
# =======================
# STEP 0: Mount Google Drive
# =======================
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# =======================
# STEP 1: Imports
# =======================
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


In [ ]:
# =======================
# STEP 2: Initialize Weights & Biases
# =======================
wandb.init(project="NASNetLarge-flowers", config={
    "epochs": 50,
    "batch_size": 16,
    "learning_rate": 0.001,
    "architecture": "NASNetLarge",
    "pretrained": True,
    "input_size": 331
})
config = wandb.config


In [ ]:
# =======================
# STEP 3: Data Preparation
# =======================
IMAGE_SIZE = (331, 331)  # Required input size for NASNetLarge

train_dir = "/content/drive/MyDrive/5flowersdata/flowers/train"
val_dir = "/content/drive/MyDrive/5flowersdata/flowers/val"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=config.batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=config.batch_size,
    class_mode='categorical'
)

Found 4008 images belonging to 5 classes.
Found 1011 images belonging to 5 classes.


In [ ]:
# =======================
# STEP 4: Load NASNetLarge Model
# =======================
base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))

# Freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# =======================
# STEP 5: Compile & Train
# =======================
model.compile(optimizer=Adam(learning_rate=config.learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    WandbMetricsLogger(log_freq="epoch"),
    WandbModelCheckpoint(filepath="model.keras", monitor="val_accuracy", save_best_only=True)
]

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=config.epochs,
    callbacks=callbacks
)

wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.6670 - loss: 0.9523 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 4283s 17s/step - accuracy: 0.6673 - loss: 0.9512 - val_accuracy: 0.8437 - val_loss: 0.4284
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4476s 18s/step - accuracy: 0.8737 - loss: 0.3576 - val_accuracy: 0.8526 - val_loss: 0.4244
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4396s 17s/step - accuracy: 0.8936 - loss: 0.2798 - val_accuracy: 0.8714 - val_loss: 0.3364
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4379s 17s/step - accuracy: 0.9277 - loss: 0.2144 - val_accuracy: 0.8833 - val_loss: 0.3319
Epoch 5/50
 32/251 ━━━━━━━━━━━━━━━━━━━━ 50:51 14s/step - accuracy: 0.9460 - loss: 0.1308